In [ ]:
import ROOT
import fastjet as fj
import fjext
import fjcontrib
import fjtools

import pythia8
import pythiafjext
import pythiaext
from heppy.pythiautils import configuration as pyconf

from tqdm.notebook import tqdm
import argparse
import os
import sys

In [ ]:
def get_args_from_settings(ssettings):
    sys.argv=[' '] + ssettings.split()
    parser = argparse.ArgumentParser(description='pythia8 fastjet on the fly')
    pyconf.add_standard_pythia_args(parser)
    parser.add_argument('--output', default="test_ang_ue.root", type=str)
    parser.add_argument('--user-seed', help='pythia seed', default=1111, type=int)
    args = parser.parse_args()
    return args

In [ ]:
mycfg = []
ssettings = "--py-ecm 5000 --py-minbias --user-seed=100000"
args = get_args_from_settings(ssettings)
pythia_mb = pyconf.create_and_init_pythia_from_args(args, mycfg)

In [ ]:
mycfg = []
ssettings = "--py-ecm 5000 --user-seed=100000 --nev 10000"
args = get_args_from_settings(ssettings)
pythia_hard = pyconf.create_and_init_pythia_from_args(args, mycfg)

In [ ]:
max_eta_hadron=1
parts_selector_h = fj.SelectorAbsEtaMax(max_eta_hadron)
jet_R0 = 0.4
jet_selector = fj.SelectorPtMin(20.0) & fj.SelectorPtMax(200.0) & fj.SelectorAbsEtaMax(max_eta_hadron - 1.05 * jet_R0)
n_pileup = 1

In [ ]:
# print the banner first
fj.ClusterSequence.print_banner()
print()
# set up our jet definition and a jet selector
jet_R0 = 0.4
jet_def = fj.JetDefinition(fj.antikt_algorithm, jet_R0)
print(jet_def)

In [ ]:
def matched(j1, j2):
    # j2i = fjtools.matched_Ry(j1, j2)
    mpt = fjtools.matched_pt(j1, j2)
    if mpt > 0.5:
        return True, j1, j2, fjext.lambda_beta_kappa(j1, 1.0, 1.0, 1.0), fjext.lambda_beta_kappa(j2, 1.0, 1.0, 1.0)
    return False

In [ ]:
def fill_matched(j1s, j2s, tj_no_pup, tj_pup, tj_delta):
    for j1 in j1s:
        tj_no_pup.Fill(j1.perp(), j1.eta(), j1.phi(), 
                       fjext.lambda_beta_kappa(j1, 1.0, 1.0, 1.0),
                       fjext.lambda_beta_kappa(j1, 2.0, 1.0, 1.0),
                       fjext.lambda_beta_kappa(j1, 3.0, 1.0, 1.0))
        for j2 in j2s:
            mpt = fjtools.matched_pt(j1, j2)
            tj_delta.Fill(j1.perp(), j1.eta(), j1.phi(), 
                          fjext.lambda_beta_kappa(j1, 1.0, 1.0, 1.0),
                          fjext.lambda_beta_kappa(j1, 2.0, 1.0, 1.0),
                          fjext.lambda_beta_kappa(j1, 3.0, 1.0, 1.0),
                          j2.perp(), j2.eta(), j2.phi(),
                          fjext.lambda_beta_kappa(j2, 1.0, 1.0, 1.0),
                          fjext.lambda_beta_kappa(j2, 2.0, 1.0, 1.0),
                          fjext.lambda_beta_kappa(j2, 3.0, 1.0, 1.0), 
                          mpt)
    for j1 in j2s:
        tj_pup.Fill(j1.perp(), j1.eta(), j1.phi(),
                    fjext.lambda_beta_kappa(j1, 1.0, 1.0, 1.0),
                    fjext.lambda_beta_kappa(j1, 2.0, 1.0, 1.0),
                    fjext.lambda_beta_kappa(j1, 3.0, 1.0, 1.0))

In [ ]:
tj_delta = ROOT.TNtuple("tj_delta", "tj_delta", "pt:eta:phi:L11:L21:L31:ptm:etam:phim:L11m:L21m:L31m:mpt")
tj_no_pup = ROOT.TNtuple("tj_no_pup", "tj_no_pup", "pt:eta:phi:L11:L21:L31")
tj_pup = ROOT.TNtuple("tj_pup", "tj_pup", "pt:eta:phi:L11:L21:L31")
hmult_hard = ROOT.TH1F("hmult_hard", "hmult_hard", 300, 0, 300)
hmult_pup = ROOT.TH1F("hmult_pup", "hmult_pup", 300, 0, 300)
hpt_acc_hard = ROOT.TProfile2D("hpt_acc_hard", "hpt_acc_hard;#eta;#varphi", 50, -1, 1, 50, 0, ROOT.TMath.Pi() * 2.)
hpt_acc_pup = ROOT.TProfile2D("hpt_acc_pup", "hpt_acc_pup;#eta;#varphi", 50, -1, 1, 50, 0, ROOT.TMath.Pi() * 2.)

In [ ]:
for n in tqdm(range(args.nev)):
    if not pythia_hard.next():
        continue
    parts_pythia_h = pythiafjext.vectorize_select(pythia_hard, [pythiafjext.kFinal], 0, False)
    parts_pythia_h_selected = parts_selector_h(parts_pythia_h)

    parts_pileup = None
    for ipile in range(n_pileup):
        while not pythia_mb.next():
            continue
        parts_pythia_h_ue = pythiafjext.vectorize_select(pythia_mb, [pythiafjext.kFinal], 10000, False)
        parts_pythia_h_selected_ue = parts_selector_h(parts_pythia_h_ue)
        if parts_pileup is None:
            parts_pileup = parts_pythia_h_selected_ue
        else:
            parts_pileup += parts_pythia_h_selected_ue

    mult_hard = len(parts_pythia_h_selected)
    mult_ue = len(parts_pileup)
    
    jets_h = fj.sorted_by_pt(jet_selector(jet_def(parts_pythia_h_selected)))
    jets_h_w_ue = fj.sorted_by_pt(jet_selector(jet_def(parts_pileup + parts_pythia_h_selected)))

    if len(jets_h) < 1:
        continue
        
    fill_matched(jets_h, jets_h_w_ue, tj_no_pup, tj_pup, tj_delta)

    hmult_hard.Fill(mult_hard)
    hmult_pup.Fill(mult_ue)

    _tmp = [hpt_acc_hard.Fill(p.eta(), p.phi(), p.perp()) for p in parts_pythia_h_selected]
    _tmp = [hpt_acc_pup.Fill(p.eta(), p.phi(), p.perp()) for p in parts_pileup]



    #_tmp = [hpt_L_nopup.Fill(j.perp(), fjext.lambda_beta_kappa(j, 1.0, 1.0, 1.0)) for j in jets_h]
    #_tmp = [hpt_L_pup.Fill(j.perp(), fjext.lambda_beta_kappa(j, 1.0, 1.0, 1.0)) for j in jets_h_w_ue]
    
    #_matched = [matched(j1, j2) for j1 in jets_h for j2 in jets_h_w_ue]
    #for _m in _matched:
    #    if _m:
    #        hdpt.Fill(_m[1].perp(), _m[1].perp() - _m[2].perp())
    #        hdL.Fill(_m[1].perp(), _m[3] - _m[4])
        
    

In [ ]:
pythia_hard.stat()

In [ ]:
pythia_mb.stat()

In [ ]:
%jsroot on
cm = ROOT.TCanvas("cm","cm",600,400)
hmult_pup.SetLineColor(3)
hmult_pup.GetXaxis().SetTitle("charged particle multiplicity")
hmult_pup.GetYaxis().SetTitle("counts")
hmult_pup.Draw()
hmult_hard.SetLineColor(2)
hmult_hard.Draw("same")
cm.SetLogy()
cm.BuildLegend()
cm.Draw()

In [ ]:
%jsroot on
c = ROOT.TCanvas("c","c",1000,800)
c.Divide(2,2)
c.cd(1)
tj_delta.Draw("dpt:pt>>hdpt", "mpt>0.5", "colz")
hdpt = ROOT.gDirectory.Get("hdpt")
hdpt.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c.cd(2)
tj_delta.Draw("dL11:pt>>hdL11", "mpt>0.5", "colz")
hdL11 = ROOT.gDirectory.Get("hdL11")
hdL11.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c.cd(3)
tj_delta.Draw("dL21:pt>>hdL21", "mpt>0.5", "colz")
hdL21 = ROOT.gDirectory.Get("hdL21")
hdL21.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c.cd(4)
tj_delta.Draw("dL31:pt>>hdL31", "mpt>0.5", "colz")
hdL31 = ROOT.gDirectory.Get("hdL31")
hdL31.ProfileX().Draw("same")
ROOT.gPad.SetLogz()


c.Draw()

In [ ]:
%jsroot on
c1 = ROOT.TCanvas("c1","c1",1000,800)
c1.Divide(2,2)
c1.cd(1)
tj_delta.Draw("dpt:pt>>hdpt", "mpt>0.5 && pt>20 && pt<40", "colz")
hdpt = ROOT.gDirectory.Get("hdpt")
hdpt.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c1.cd(2)
tj_delta.Draw("dL11:L11>>pthdL11", "mpt>0.5 && pt>20 && pt<40", "colz")
pthdL11 = ROOT.gDirectory.Get("pthdL11")
pthdL11.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c1.cd(3)
tj_delta.Draw("dL21:L21>>pthdL21", "mpt>0.5 && pt>20 && pt<40", "colz")
pthdL21 = ROOT.gDirectory.Get("pthdL21")
pthdL21.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c1.cd(4)
tj_delta.Draw("dL31:L31>>pthdL31", "mpt>0.5 && pt>20 && pt<40", "colz")
pthdL31 = ROOT.gDirectory.Get("pthdL31")
pthdL31.ProfileX().Draw("same")
ROOT.gPad.SetLogz()


c1.Draw()

In [ ]:
%jsroot on
c2 = ROOT.TCanvas("c2","c2",1000,800)
c2.Divide(2,2)
c2.cd(1)
tj_delta.Draw("dpt:pt>>hdpt", "mpt>0.5 && pt>60 && pt<80", "colz")
hdpt = ROOT.gDirectory.Get("hdpt")
hdpt.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c2.cd(2)
tj_delta.Draw("dL11:L11>>pt2hdL11", "mpt>0.5 && pt>60 && pt<80", "colz")
pt2hdL11 = ROOT.gDirectory.Get("pt2hdL11")
pt2hdL11.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c2.cd(3)
tj_delta.Draw("dL21:L21>>pt2hdL21", "mpt>0.5 && pt>60 && pt<80", "colz")
pt2hdL21 = ROOT.gDirectory.Get("pt2hdL21")
pt2hdL21.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c2.cd(4)
tj_delta.Draw("dL31:L31>>pt2hdL31", "mpt>0.5 && pt>60 && pt<80", "colz")
pt2hdL31 = ROOT.gDirectory.Get("pt2hdL31")
pt2hdL31.ProfileX().Draw("same")
ROOT.gPad.SetLogz()


c2.Draw()

In [ ]:
%jsroot on
c3 = ROOT.TCanvas("c3","c3",1000,800)
c3.Divide(1,2)
c3.cd(1)
hpt_acc_pup.Draw("colz")
c3.cd(2)
hpt_acc_hard.Draw("colz")
c3.Draw()

In [ ]:
%jsroot on
tc = ROOT.TCanvas()
tj_delta.Draw("dpt:pt", "mpt>0.5", "colz")
ROOT.gPad.SetLogz()
tc.Draw()

In [ ]:
%jsroot on
tcL = ROOT.TCanvas()
tcL.Divide(2,2)
tcL.cd(1)
tj_delta.Draw("dL11:L11>>pt2hdL11x", "mpt>0.5 && pt>60 && pt<80", "colz")
pt2hdL11x = ROOT.gDirectory.Get("pt2hdL11x")

tj_delta.Draw("dL11:L11>>pt2hdL11xx", "mpt>0.5 && pt>60 && pt<80", "colz")
pt2hdL11xx = ROOT.gDirectory.Get("pt2hdL11xx")

pxx = pt2hdL11xx.ProjectionX()
pxx.SetLineStyle(2)
pxx.SetFillColorAlpha(2, 0.5)
pxx.SetFillStyle(1001)
pxx.Draw()


tcL.cd(2)
pyy = pt2hdL11xx.ProjectionY()
pyy.SetLineStyle(2)
pyy.SetFillColorAlpha(3, 0.5)
pyy.SetFillStyle(1001)
pyy.Draw()




#pt2hdL11x.ProjectionX().Draw()
#pxx.Draw("same")



ROOT.gPad.SetLogy()
ROOT.gPad.BuildLegend()
tcL.Draw()
